In [39]:
import networkx as nx
import numpy as np
from fractions import Fraction
from decimal import Decimal

class PlumbingGraph(nx.Graph):
    def __init__(self):
        """Instantiates a new PlumbingGraph which is a subclass of nx.Graph
        
        """
        self.chains = []
        self.total = 0
        super(PlumbingGraph,self).__init__()
        
    def add_node(self, node_for_adding, **attr):
        """Add a node to the graph
            
            Args:
                node_for_adding: The node to be added
                **attr: Optional dict within a dict of attributes for the nodes.
        """
        self.total += 1
        nx.Graph(self).add_node(node_for_adding, **attr)

        
    def total_area(self):
        total = 0
        for n in nodes:
            total += n.area
            
    def total_genus(self):
        total_genus = 0
        for n in nodes:
            total_genus += n.genus
    
    def attach_chain_to(self, node, chain):
        """Attaches a chain to a specified node
        
            Args:
               node (PlumbingNode): The node that we are attaching to
               chain (Chain): A plumbing chain to attach
        """
        
        # Add all the edges and nodes
        self.add_nodes_from(chain.nodes)
        self.add_edges_from(chain.edges)
        
        # Connect the head of the chain to node
        self.add_edge(node,chain.head)
        
        # Add the new chain to the chain bank
        self.chains.append(chain)
        
    def attach_chain_between(self, node1, node2, chain):
        """Attaches a chain between two specified nodes
        
            Args:
                node1 (PlumbingNode): The node which we will attach the head of the chain to
                node2 (PlumbingNode): The node which we will attach the tail of the chain to
        """
        
        # Add all the edges and nodes
        self.add_nodes_from(chain.nodes)
        self.add_edges_from(chain.edges)
        
        # Connect the head and tail of the chain to the respective nodes
        self.add_edge(node1, chain.head)
        self.add_edge(node2, chain.tail)
        
        # Add the new chain to the chain bank
        self.chains.apppend(chain)
        
    def decorate_node(self, node, euler_number: int, genus: int, area: float):
        self.nodes[node]["e"] = euler_number
        self.nodes[node]["g"] = genus
        self.nodes[node]["a"] = area


        
            
class Chain(PlumbingGraph):
    def __init__(self, components: [int], area: [float]):
        """ Instantiates a new Chain from input data
        
            Args:
                components ([int]): A list of integer components for the chain
                area ([float]): A list of symplectic area information for each node
        """
        
        # Check if the input data is valid
        if len(area) != len(components):
            raise Exception("Area vector does not match the size of the arm.")
        
        super(Chain,self).__init__()
        
        self.order_function = []
        self.index_lookup = {}
        self.length = len(components)
        
        # Attach nodes in a linear fashion keeping track of order and indices
        for i,k in enumerate(components):
            
            # Create and add a new node to the chain
            self.total += 1
            self.add_node(self.total, attr={"euler_number":k, "genus":0, "area":area[i]})
            
            # The first node is the head
            if i == 0:
                self.head = self.total
                
            # The last node is the tail
            elif i == self.length - 1:
                self.tail = self.total
                self.add_edge(self.total - 1, self.total)
            
            # Otherwise, we just attach the new node
            else:
                self.add_edge(self.total - 1, self.total)
            
            # Set the index and ordering information
            self.order_function.append(self.total)
            self.index_lookup[self.total] = i

        
        # Compute the associated continued fraction for the chain
        self.associated_rational = compute_continued_fraction(components)
        
    
    def update_order_function(self, i: int, k: int, deletion: bool):
        """Updates the order function for deletion or insertion of k nodes
        """
        ind = (i + 1) + k
        
        
        if deletion == True:
            end_nodes = self.order_function[ind:self.length]
            marked_for_deletion = self.order_function[i+1, ind]
            
            for j,n in enumerate(end_nodes):
                self.order_function[i+1+j] = n
                self.index_lookup[n] -= k
            
            for n in marked_for_deletion:
                del self.index_lookup[n]
            
            del self.order_function[self.length - k: self.length]
            self.length -= k
            
        else:
            to_shift = []
            if i+1 < self.length:
                # If we have to shift nodes over, save and remove them
                to_shift = self.order_function[i+1:self.length]
                del self.order_function[i+1:self.length]          
            
            # Shift the elements (if any) after extending k elements                 
            self.order_function.extend(range(k))
            self.order_function.extend(to_shift)
            
            # Update the indices and the length 
            for n in to_shift:
                self.index_lookup[n] += k
            
            self.length += k
    
    
    def insert_k_nodes_at(self, i: int, k: int, new_nodes, **attr):
        
        # Add the new nodes and remove the old edge
        self.add_nodes_from(new_nodes)
        self.remove_edge(self.order_function[i], self.order_function[i+1])
        
        # Prepare the order function for insertion of k nodes
        self.update_order_function(i, k, deletion=False)
        
        # Set the order of the new nodes and add edges
        
        self.add_edge(self.order_function[i], self.order_function[i+1])
        for j,n in enumerate(new_nodes):
                self.order_function[i+1+j] = n
                self.index_lookup[n] = i+j
                
                self.add_edge(self.order_function[i + j], n)
        
        self.add_edge(self.order_function[i +k],self.order_function[i + 1 + k])
        
        
    def insert_node_at(self, i: int, new_node, **attr):
        self.insert_k_nodes_at(i, 1, [new_node], attr=attr)
    
    def delete_k_nodes_from(self, i: int, k: int):
        old_nodes = self.nodes[i:i+k]
        
        self.update_order_function(i, k, deletion=True)  
        self.delete_nodes_from(old_nodes)
        
        if i-1 < 0 or i+k == self.length:
            pass
        else:
            self.add_edge(self.order_function[i-1], self.order_function[i+k])
            

    
    def blow_up(self, i: int, area: float):
        """Perform a combinatorial -1 blow-up on the edge between node i and node i+1
        
            Args:
                node1 (PlumbingNode): One node connected to the edge
                node2 (PlumbingNode): The other node connected to the edge
        """
        
        # Check if i is in the correct range
        if i >= self.length - 1 or i < 0:
            raise Exception("Index out of range!")
        
        
        # Add the new node
        new_node = self.size
        self.insert_node_at(i, new_node, attr={"euler_number":-1, "genus":0, "area":area})
        
        # Update Euler numbers
        self.order_function[i]["euler_number"] -= 1
        
        if i != self.length - 1:
            self.order_function[i+2]["euler_number"] -= 1
        
        
    def transfer_right(self, i: int, k: int):
        
        assert(i < self.length and i >= 0)
    
        # We can only transfer at a 0-framed node
        assert(self.order_function[i].euler_number == 0)
        
        if i == 0:
            nieghbor_set = set(self.neighbors(self.head)) - {self.order_function[i+1]} 
            for n in neighbor_set:
                n.euler_number -= k
                
            self.order_function[i+1].euler_number += k
        elif i == self.length - 1:
            if len(self.neighbors(self.tail)) == 1:
                self.order_function[self.length - 2]["euler_number"] -= k
            else:
                self.order_function[i-1] -= k
                neighbor_set = set(self.neighbors(self.tail)) - {self.order_function[i-1]}
                for n in neighbor_set:
                    n.euler_number += k
        else:
            self.order_function[i-1].euler_number -= k
            self.order_function[i+1].euler_number += k
     
    def transfer_left(self, i: int, k: int):
        self.transfer_right(i, -k)
    
    def slide_far_left(self, i: int):
        
        assert(i < self.length and i >= 0)
    
        # We can only slide a 0-0 configuration
        assert(self.order_function[i].euler_number == 0 and self.order_function[i+1].euler_number == 0)
        
        
        
        
        
        
    def __iter__(self):
        self.iteration = 0
        
    def __next__(self):
        if self.iteration < self.length:
            self.iteration += 1
            return order_function[self.iteration - 1]
        else:
            raise StopIteration
            
    def __str__(self):
        out = ""
        for i in range(self.length):
            out += str(self.order_function[i]) + ', '
            
        return out
            

def compute_continued_fraction(components: [int]) -> Fraction:
    """Compute the continued fraction expansion associated to an integer list of components
    
        Args:
            components ([int]): a list of the integer components of the continued fraction
            
        Return:
            A Fraction which is the reduced version of the associated rational number
    """
    if len(components) == 1:
        return Fraction(components[0],1)
    
    return Fraction(components[0],1) - 1/compute_continued_fraction(components[1:])




G = PlumbingGraph()

root_node = 0
G.add_node(root_node, attr={"euler_number":7, "genus": 2, "area": .45})

arms = [
    [5,1,3],
    [4,1,6,7,8],
    [3,1,5,7,4,4,6,78,8,4],
    [2,1,5,7,8,4]
]

areas = [
    [0.5,0.5,0.5],
    [.78,.45,.345,.45,.89],
    [.56, .78, .543, .67, .46, .54, .45, .67, .565, .45],
    [.2, .2, .45, .5, .5, .45]
]

for i,arm in enumerate(arms):
    c = Chain(arm, areas[i])
    print(c)
    
    c.blow_up(1,100)
    print("Blown up is: ")
    print(c)
    
    c.transfer_right(1,2)
    print("Transferred is: ")
    print(c)
    
    r = c.associated_rational
    print("This arm is the space L({0},{1})".format(r.numerator,r.denominator))

    G.attach_chain_to(root_node, c)

                 
# Plot the result
edge_nodes = set(G) - {root_node}
pos = nx.spring_layout(G.subgraph(edge_nodes))
pos[root_node] = np.array([0, 0])  
nx.draw(G, pos, with_labels=True)






2, 4, 6, 


NetworkXError: The edge 4-6 is not in the graph